In [1]:
%%configure
{"conf": {
    "spark.app.name":"oh-my-git_final"
}}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7611,application_1589299642358_2106,pyspark,idle,Link,Link,
7632,application_1589299642358_2126,pyspark,idle,Link,Link,
7633,application_1589299642358_2127,pyspark,idle,Link,Link,
7644,application_1589299642358_2140,pyspark,idle,Link,Link,
7670,application_1589299642358_2166,pyspark,busy,Link,Link,
7674,application_1589299642358_2170,pyspark,idle,Link,Link,
7676,application_1589299642358_2172,pyspark,busy,Link,Link,
7677,application_1589299642358_2173,pyspark,dead,Link,Link,
7681,application_1589299642358_2177,pyspark,idle,Link,Link,
7683,application_1589299642358_2179,pyspark,idle,Link,Link,


In [2]:
# Initialization

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7702,application_1589299642358_2198,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from geopy import distance
import pyspark.sql.types as T
import pyspark.sql.functions  as F
from pyspark.sql.window import Window
import datetime
import networkx as nx
import pandas as pd
zurichHB = (47.378177, 8.540192)
cleveland_oh = (49.378177, 9.540192)
print(distance.distance(zurichHB, cleveland_oh).m)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

234402.47499

In [4]:
stop_times_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/stop_times")
stops_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/stops")
trips_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/trips")
calendar_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/calendar")
routes_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/routes")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

First let's filter stations in 15km radius of Zurich HB

In [5]:
@F.udf(T.DoubleType())
def getDistToZurich(lat,lon):
    zurichHB = (47.378177, 8.540192)
    geo = (lat,lon)
    return distance.distance(zurichHB, geo).km
stops_inradius_df= stops_df.withColumn("dist",getDistToZurich("stop_lat","stop_lon")).filter("dist <= 15")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Then get the trips of these stops

In [9]:
stops_inradius_df = stops_inradius_df.drop("dist")
#Join the stops and trips
stops_joined_df = stop_times_df.join(stops_inradius_df,stop_times_df.stop_id == stops_inradius_df.stop_id ).drop(stops_inradius_df.stop_id)
stops_joined_df.drop("parent_station","location_type","pickup_type","drop_off_type").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+---------------+-------------+------------------+----------------+----------------+
|             trip_id|arrival_time|departure_time|        stop_id|stop_sequence|         stop_name|        stop_lat|        stop_lon|
+--------------------+------------+--------------+---------------+-------------+------------------+----------------+----------------+
|5.TA.1-1-A-j19-1.3.H|    02:42:00|      02:42:00|    8503305:0:3|            2|        Effretikon|  47.42593043029|8.68666388288076|
|5.TA.1-1-A-j19-1.3.H|    02:46:00|      02:46:00|    8503306:0:3|            3|         Dietlikon|47.4203145327083|8.61925430395105|
|5.TA.1-1-A-j19-1.3.H|    02:50:00|      02:50:00|    8503147:0:1|            4|         Stettbach| 47.397334167601|8.59613166853459|
|5.TA.1-1-A-j19-1.3.H|    02:55:00|      02:55:00|    8503003:0:1|            5|Zürich Stadelhofen|47.3667936853425|8.54846705955257|
|5.TA.1-1-A-j19-1.3.H|    02:58:00|      03:00:00|8503000:0:41

In [10]:
#Write this to use it later
#stops_joined_df.write.parquet("user/boecuego/stops_joined_df.parquet")

stops_joined_df = spark.read.parquet("user/boecuego/stops_joined_df.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Create Stop Connetions via Trips 

Here we are creating all possible stop connections reachable by public transports. For example, if we have stops_ids 1,2,3,4 for trip X, then connections are 1-2,1-3,1-4,2-3,2-4,3-4 (in other words combinations)

In [11]:
#First Collect all relevant information for each trip_id

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df_grouped = stops_joined_df.groupBy("trip_id").agg(F.collect_list("stop_id").alias("stop_ids"),F.collect_list("stop_sequence").alias("stop_sequences"),\
                                      F.collect_list("arrival_time").alias("arrival_times"),\
                                      F.collect_list("departure_time").alias("departure_times"),\
                                      )                                    

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Then create combinations of these information 


In [16]:
import itertools
@F.udf(T.ArrayType(
       T.ArrayType(T.StructType([
    T.StructField("pair1", T.StringType(), False),
    T.StructField("pair2", T.StringType(), False)
]))))
def pairs(stop_ids,stop_sequences,arrival_times,departure_times):
    stop_id_pairs = list(itertools.combinations(stop_ids,2))
    stop_sequence_pairs = list(itertools.combinations(stop_sequences,2))
    arrival_time_pairs = list(itertools.combinations(arrival_times,2))
    departure_time_pairs = list(itertools.combinations(departure_times,2))
    return [stop_id_pairs,stop_sequence_pairs,arrival_time_pairs,departure_time_pairs]
    

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
df_grouped.cache()
df_grouped.take(1)
df_grouped.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

173184

In [18]:
df_pairs = df_grouped.withColumn("all_pairs",pairs("stop_ids","stop_sequences","arrival_times","departure_times"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df_pairs.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             trip_id|            stop_ids|      stop_sequences|       arrival_times|     departure_times|           all_pairs|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|1.TA.26-18-j19-1.1.H|[8591315, 8591142...|[1, 2, 3, 4, 5, 6...|[28:45:00, 28:46:...|[28:45:00, 28:46:...|[[[8591315, 85911...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 1 row

In [24]:
df_pairs_sep = df_pairs.withColumn("stop_id_pairs",F.col("all_pairs")[0])\
                        .withColumn("stop_sequence_pairs",F.col("all_pairs")[1])\
                        .withColumn("arrival_time_pairs",F.col("all_pairs")[2])\
                        .withColumn("departure_time_pairs",F.col("all_pairs")[3])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

For example this is how it looks for stop_id pairs, same applies for other columns.

In [29]:
df_pairs_sep.select("trip_id",F.explode("stop_id_pairs").alias("stop_id_pair")).withColumn("rown",F.monotonically_increasing_id()).show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+----+
|             trip_id|      stop_id_pair|rown|
+--------------------+------------------+----+
|1.TA.26-18-j19-1.1.H|[8591315, 8591142]|   0|
|1.TA.26-18-j19-1.1.H|[8591315, 8530811]|   1|
|1.TA.26-18-j19-1.1.H|[8591315, 8591104]|   2|
|1.TA.26-18-j19-1.1.H|[8591315, 8591429]|   3|
|1.TA.26-18-j19-1.1.H|[8591315, 8591180]|   4|
|1.TA.26-18-j19-1.1.H|[8591315, 8530812]|   5|
|1.TA.26-18-j19-1.1.H|[8591315, 8591364]|   6|
|1.TA.26-18-j19-1.1.H|[8591315, 8530813]|   7|
|1.TA.26-18-j19-1.1.H|[8591142, 8530811]|   8|
|1.TA.26-18-j19-1.1.H|[8591142, 8591104]|   9|
+--------------------+------------------+----+
only showing top 10 rows

In [30]:
sti = df_pairs_sep.select("trip_id",F.explode("stop_id_pairs").alias("stop_id_pair")).withColumn("rown",F.monotonically_increasing_id()).alias("sti")
ssp = df_pairs_sep.select(F.explode("stop_sequence_pairs").alias("stop_sequence_pair"),F.monotonically_increasing_id().alias("rown")).alias("ssp")
atp = df_pairs_sep.select(F.explode("arrival_time_pairs").alias("arrival_time_pair"),F.monotonically_increasing_id().alias("rown")).alias("atp")
dtp = df_pairs_sep.select(F.explode("departure_time_pairs").alias("departure_time_pair"),F.monotonically_increasing_id().alias("rown")).alias("dtp")



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
#Uncomment this to run faster later

#ti.cache()
#ssp.cache()
#atp.cache()
#dtp.cache()
#sti.count()
#ssp.count()
#atp.count()
#dtp.count()

#Uncomment this to run faster later

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
joined = sti.join(ssp,sti.rown == ssp.rown).drop(ssp.rown).join(atp,sti.rown == atp.rown).drop(atp.rown).join(dtp,sti.rown == dtp.rown).drop(dtp.rown)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
#joined.write.mode("overwrite").parquet("user/boecuego/df_all_stop_connections.parquet")
joined = spark.read.parquet("user/boecuego/df_all_stop_connections.parquet")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Now let's see the final table

In [40]:
joined.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+----+------------------+--------------------+--------------------+
|             trip_id|        stop_id_pair|rown|stop_sequence_pair|   arrival_time_pair| departure_time_pair|
+--------------------+--------------------+----+------------------+--------------------+--------------------+
|1.TA.26-18-j19-1.1.H|  [8591142, 8591429]|  10|            [2, 5]|[28:46:00, 28:50:00]|[28:46:00, 28:50:00]|
|1005.TA.26-131-j1...|[8503855:0:F, 858...| 120|            [1, 7]|[16:14:00, 16:21:00]|[16:14:00, 16:21:00]|
|1022.TA.26-33E-j1...|  [8591230, 8591196]| 423|            [6, 7]|[18:44:00, 18:46:00]|[18:44:00, 18:46:00]|
|1022.TA.26-33E-j1...|  [8576198, 8576182]| 485|          [14, 17]|[18:55:00, 18:59:00]|[18:55:00, 18:59:00]|
|103.TA.26-1-A-j19...|[8503512:0:2, 850...| 500|          [14, 17]|[02:51:00, 03:00:00]|[02:51:00, 03:00:00]|
+--------------------+--------------------+----+------------------+--------------------+--------------------+
only showi

#### Filter for weekdays

Now we need to filter rows to obtain trips that are available Only on Weekdays (monday,tuesday,wednesday,thursday,friday)

In [46]:
stop_times_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/stop_times")
stops_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/stops")
trips_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/trips")
calendar_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/calendar")
routes_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/routes")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
df_service  =joined.join(trips_df,trips_df.trip_id ==joined.trip_id  )
df_days = df_service.join(calendar_df,calendar_df.service_id == df_service.service_id)
df_weekdays = df_days.filter("monday == 'true' AND tuesday == 'true' AND wednesday == 'true' AND thursday == 'true' AND friday == 'true' ")
cols = joined.columns
df_weekdays = df_weekdays.drop(trips_df.trip_id)
joined_filtered  = df_weekdays.select(*cols)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
#joined_filtered.write.mode("overwrite").parquet("user/boecuego/df_all_stop_connections_filtered.parquet")
joined_filtered = spark.read.parquet("user/boecuego/df_all_stop_connections_filtered.parquet")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
#Sort dataframe and selet only interested columns
joineds = joined_filtered.sort("rown").select("trip_id",F.col("stop_id_pair").pair1.alias("stop_id1")\
                        ,F.col("stop_id_pair").pair2.alias("stop_id2")\
                        ,F.col("stop_sequence_pair").pair1.alias("stop_sequence1")\
                        ,F.col("stop_sequence_pair").pair2.alias("stop_sequence2")\
                        ,F.col("arrival_time_pair").pair1.alias("arrival_time1")\
                        ,F.col("arrival_time_pair").pair2.alias("arrival_time2")\
                        ,F.col("departure_time_pair").pair1.alias("departure_time1")\
                        ,F.col("departure_time_pair").pair2.alias("departure_time2"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Finally we have all reachable stops

In [57]:
joineds.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+--------+--------------+--------------+-------------+-------------+---------------+---------------+
|             trip_id|   stop_id1|stop_id2|stop_sequence1|stop_sequence2|arrival_time1|arrival_time2|departure_time1|departure_time2|
+--------------------+-----------+--------+--------------+--------------+-------------+-------------+---------------+---------------+
|1005.TA.26-131-j1...|8503855:0:F| 8589111|             1|             2|     16:14:00|     16:15:00|       16:14:00|       16:15:00|
|1005.TA.26-131-j1...|8503855:0:F| 8573553|             1|             3|     16:14:00|     16:16:00|       16:14:00|       16:16:00|
|1005.TA.26-131-j1...|8503855:0:F| 8573554|             1|             4|     16:14:00|     16:18:00|       16:14:00|       16:18:00|
|1005.TA.26-131-j1...|8503855:0:F| 8573555|             1|             5|     16:14:00|     16:19:00|       16:14:00|       16:19:00|
|1005.TA.26-131-j1...|8503855:0:F| 8588985|             1|    

# Create Stop Connections via Walking

Get all walkable stop pair

**Conditions:**
- If distance(stop1,stop2) < 500 meters
- If stop in radius (15km from Zurich)

**Walking speed :** 50 meters in 1 minute

In [63]:
@F.udf(T.DoubleType())
def getDist(lat1,lon1,lat2,lon2):
    geo1 = (lat1,lon1)
    geo2 = (lat2,lon2)
    dist = distance.distance(geo1, geo2).m
    return dist

stops_inradius_df = stops_inradius_df.drop("dist")

stopDist = stops_inradius_df.alias("stops1").crossJoin(stops_inradius_df.alias("stops2"))\
.where(getDist("stops1.stop_lat","stops1.stop_lon","stops2.stop_lat","stops2.stop_lon")<= 500)\
.withColumn("dist",getDist("stops1.stop_lat","stops1.stop_lon","stops2.stop_lat","stops2.stop_lon"))


#Other option is this but don't know if it is way more efficient or more or less similiar
#stopDistFiltered = stops_inradius_df.alias("stops1").join(stops_inradius_df.alias("stops2")\
#,getDist("stops1.stop_lat","stops1.stop_lon","stops2.stop_lat","stops2.stop_lon")<=500)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
#Rename columns
walk_stops = stopDist.select(F.col("stops1.stop_id").alias("stop1_id")\
               ,F.col("stops1.stop_name").alias("stop1_name")\
               ,F.col("stops1.stop_lat").alias("stop1_lat")\
               ,F.col("stops1.stop_lon").alias("stop1_lon")\
               ,F.col("stops2.stop_id").alias("stop2_id")\
               ,F.col("stops2.stop_name").alias("stop2_name")\
               ,F.col("stops2.stop_lat").alias("stop2_lat")\
               ,F.col("stops2.stop_lon").alias("stop2_lon")\
               ,F.col("dist"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#walk_stops.write.parquet("user/boecuego/walk_stops.parquet")

walk_stops = spark.read.parquet("user/boecuego/walk_stops.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
#Dist is distance between two stops
walk_stops.drop("stop1_lat","stop1_lon","stop2_lat","stop2_lon").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+-----------+--------------------+------------------+
|stop1_id|          stop1_name|   stop2_id|          stop2_name|              dist|
+--------+--------------------+-----------+--------------------+------------------+
| 8500926|Oetwil a.d.L., Sc...|    8500926|Oetwil a.d.L., Sc...|               0.0|
| 8500926|Oetwil a.d.L., Sc...|    8590616|Geroldswil, Schwe...|122.61607002692374|
| 8500926|Oetwil a.d.L., Sc...|    8590737|Oetwil an der Lim...|  300.671189772556|
| 8502186|Dietikon Stoffelbach|    8502186|Dietikon Stoffelbach|               0.0|
| 8502186|Dietikon Stoffelbach|8502186:0:1|Dietikon Stoffelbach| 6.761029676373361|
+--------+--------------------+-----------+--------------------+------------------+
only showing top 5 rows

# Create Graph 

In [4]:
df_trip_network_reasonable_main = spark.read.parquet("user/boecuego/df_all_stop_connections_filtered.parquet")
walk_stops_main = spark.read.parquet("user/boecuego/walk_stops.parquet")
df_trip_network_reasonable = df_trip_network_reasonable_main
walk_stops = walk_stops_main

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df_trip_network_reasonable = df_trip_network_reasonable.sort("rown").select("trip_id",F.col("stop_id_pair").pair1.alias("stop_id1")\
                        ,F.col("stop_id_pair").pair2.alias("stop_id2")\
                        ,F.col("stop_sequence_pair").pair1.alias("stop_sequence1")\
                        ,F.col("stop_sequence_pair").pair2.alias("stop_sequence2")\
                        ,F.col("arrival_time_pair").pair1.alias("arrival_time1")\
                        ,F.col("arrival_time_pair").pair2.alias("arrival_time2")\
                        ,F.col("departure_time_pair").pair1.alias("departure_time1")\
                        ,F.col("departure_time_pair").pair2.alias("departure_time2"))
joineds = df_trip_network_reasonable

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Calculate the cost between two trip nodes:** (Arrival time of stop 2) - (Departure time of stop 1)

In [6]:
timeDiff = (F.unix_timestamp('arrival_time2', format="HH:mm:ss")
            - F.unix_timestamp('departure_time1', format="HH:mm:ss"))
df_trip_network_reasonable = df_trip_network_reasonable.withColumn("cost",F.ceil(timeDiff/60))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Calculate the cost between two walking nodes:** Distance/50 + 2minutes transfer delay

In [7]:
walk_stops = walk_stops.withColumn("cost", F.ceil(F.col("dist")/50)+2)
walk_stops = walk_stops.select(F.col('stop1_id').alias('stop_id1'), F.col('stop2_id').alias('stop_id2'), 'cost')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df_trip_network_reasonables =  df_trip_network_reasonable.select("stop_id1","stop_id2","cost")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df_trip_network_reasonables=df_trip_network_reasonables.withColumn("type",F.lit("trip"))
walk_stops = walk_stops.withColumn("type",F.lit("walk"))
unioned_df = df_trip_network_reasonables.union(walk_stops)
unioned_df = unioned_df.dropna()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Asumption for simplicity:** If there are multiple edges between two nodes we only get the edge with minimum

In [10]:
unioned_df_unique = unioned_df.groupBy("stop_id1","stop_id2").agg(F.min("cost").alias("cost"),F.collect_set("type").alias("types"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Asumption:** We want to penalize using too many transfers so we add extra 10minutes of cost to each edge

- A --> B --> C
- A --> C

we prioritize A -->C
We are also doing this because these two routes may belong to same trip and we don't want the intermediate stops

In [11]:
unioned_df_unique = unioned_df_unique.withColumn("cost",F.col("cost")+10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
pd_df = unioned_df_unique.toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
import networkx as nx
import pandas as pd

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Create the grapgh with networkx, we store cost(journey duration in minutes + penalty) and types(walking or public transport) in the edges

In [14]:
mygraph = nx.from_pandas_edgelist(pd_df, 'stop_id1', 'stop_id2',["cost","types"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Find the shortest path

In [15]:
#Find the k shortest paths
from itertools import islice
def k_shortest_paths(G, source, target, k, weight):
    return list(islice(nx.shortest_simple_paths(G, source, target, weight=weight), k))



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Find **k** shortest path between two stops

In [16]:
src = "8503000:0:41/42"
tgt  ="8591049"
#k = 10

paths = k_shortest_paths(mygraph, src, tgt,10,weight = "cost")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:

#This is a helper to get pairs in a list
from itertools import tee, izip
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return izip(a, b)
#When converting a date to string this is a helper
def append_zero(x):
    if len(str(x)) == 1:
        return "0"+str(x)
    return str(x)
#Helper to remove adjecent walks
def remove_adjecent_walks(paths):
    no_adjecentwalk =[]
    for pr in paths:
        check = True
        for p1,p2 in pairwise(pairwise(pr)):

            if mygraph[p1[0]][p1[1]]["types"] == ["walk"] and mygraph[p2[0]][p2[1]]["types"] == ["walk"]:
                #print(counter)
                check = False

        if check:
            no_adjecentwalk.append(pr)
    return no_adjecentwalk

#Helper to reverse route
def get_reverse_route(paths):
    reversed_routes  = []
    for route in paths:
        lst = []
        for a in pairwise(route):
            lst.append(a)
        lst_reversed = lst[::-1]
        reversed_routes.append(lst_reversed)
    return reversed_routes

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We don't want two consecutive walks, so filter them if any

In [20]:
no_adjecentwalk = remove_adjecent_walks(paths)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Since we know arrival time we need to **reverse** the path

In [21]:
reversed_routes = get_reverse_route(no_adjecentwalk)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
joineds.cache()
joineds.take(1)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(trip_id=u'1005.TA.26-131-j19-1.9.H', stop_id1=u'8503855:0:F', stop_id2=u'8589111', stop_sequence1=u'1', stop_sequence2=u'2', arrival_time1=u'16:14:00', arrival_time2=u'16:15:00', departure_time1=u'16:14:00', departure_time2=u'16:15:00')]

In [31]:
#This is arrival_time
current_time = datetime.datetime(2019,1,1,12,30,0,0)
cut_off_time = datetime.datetime(2019,1,1,11,50,0,0)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Algorihm:** Starting from the last connection, we try to find most suitable  trips(if not possible walks) by comparing from timetables matching the arrival time and backtrace from there

In [48]:

def find_possible_routes(reversed_routes,current_timez,cut_off_time):
    
    shorts = []
    longs = []
    for route in reversed_routes:
        current_time = current_timez
        current_trip_id = ""
        
        route_with_timetable = []
        trip = []
        prints  = []
        for p1,p2 in route:
            #p2 arrival
            #p1 departure
            if mygraph[p1][p2]["types"] == ["walk"]:

                prints.append(("after_walk",current_time))
                cost = mygraph[p1][p2]["cost"]
                current_time  = current_time - datetime.timedelta(minutes=cost-10)#-2
                prints.append(("before_walk",current_time))
                current_trip_id=""
                prints.append((p1,p2))
                continue
            else:
                #Get current times hour and minute
                hour = current_time.hour
                minute = current_time.minute

                #Convert it to string
                strtime = append_zero(hour)+":"+append_zero(minute)+":00"

                #Get closest trip to current time with matching stop_ids
                if current_trip_id != "":
                    trip = joineds.filter(F.col("trip_id") == current_trip_id)\
                    .filter("stop_id1 == '"+p1+"'"+" AND "+"stop_id2 == '"+p2+"'").take(1)
               #trip = df_trip_network_reasonable2.filter("prev_stop_id == '"+p1+"'"+" AND "+"stop_id == '"+p2+"'").filter("arrival_time <='"+strtime+"'").sort(F.desc("arrival_time")).take(1)[0]

                #If previous trip_id was different from this trip's id then put 2 minute waiting time
                if trip == [] or current_trip_id == "":

                    #2 min waiting
                    if current_trip_id != "":
                        current_time = current_time - datetime.timedelta(minutes = 2)

                    #Get hours, minutes
                    hour = current_time.hour
                    minute = current_time.minute
                    strtime = append_zero(hour)+":"+append_zero(minute)+":00"

                    #With new current time get new trip
                    trip = joineds.filter("stop_id1 == '"+p1+"'"+" AND "+"stop_id2 == '"+p2+"'").filter("arrival_time2 <='"+strtime+"'").sort(F.desc("arrival_time2")).take(1)
                    #update current trip_id
                try:
                    trip = trip[0]
                except:
                    print("Encountered a route that is not possible currently")
                    break
                current_trip_id  = trip.trip_id

                #Update current time
                prints.append((trip.trip_id,trip.arrival_time1,trip.departure_time1,trip.arrival_time2,trip.departure_time2))
                hms = trip.departure_time1.split(":")
                h = int(hms[0])
                m = int(hms[1])
                current_time = datetime.datetime(2019,1,1,h,m,0,0)

            prints.append((p1,p2))
        if current_time >= cut_off_time:
            shorts.append(prints)
            shorts.append("---------------------------------------------------------------")
        else:
            longs.append(prints)
            longs.append("---------------------------------------------------------------")
    return shorts,longs

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
good_routes,bad_routes = find_possible_routes(reversed_routes,current_time,cut_off_time)



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Encountered a route that is not possible currently

In [50]:
dashes = ""
for route in good_routes:
    print(dashes)
    dashes = ""
    for s in route:
        if s != "-":
            print(s)
        else:
            dashes+="-"
        

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


(u'1914.TA.26-11-A-j19-1.27.R', u'12:01:00', u'12:01:00', u'12:24:00', u'12:24:00')
(u'8587349', '8591049')
('after_walk', datetime.datetime(2019, 1, 1, 12, 1))
('before_walk', datetime.datetime(2019, 1, 1, 11, 56))
('8503000:0:41/42', u'8587349')

---------------------------------------------------------------
(u'1914.TA.26-11-A-j19-1.27.R', u'12:03:00', u'12:03:00', u'12:24:00', u'12:24:00')
(u'8591379', '8591049')
('after_walk', datetime.datetime(2019, 1, 1, 12, 3))
('before_walk', datetime.datetime(2019, 1, 1, 11, 55))
('8503000:0:41/42', u'8591379')

---------------------------------------------------------------
(u'1914.TA.26-11-A-j19-1.27.R', u'12:15:00', u'12:15:00', u'12:24:00', u'12:24:00')
(u'8580449', '8591049')
('after_walk', datetime.datetime(2019, 1, 1, 12, 15))
('before_walk', datetime.datetime(2019, 1, 1, 12, 11))
(u'8503006:0:7', u'8580449')
(u'235.TA.26-15-j19-1.41.H', u'11:50:00', u'11:52:00', u'11:58:00', u'11:59:00')
('8503000:0:41/42', u'8503006:0:7')

---------